In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).appName("HW_1_RysinN").getOrCreate()

In [4]:
!hdfs dfs -get /labs/laba01/ data

In [5]:
# Вариант 36
film_id = 22

In [6]:
data_rdd = sc.textFile("hdfs:///labs/laba01/ml-100k/u.data")\
                .map(lambda line: line.split("\t"))\
                .map(lambda line: (line[1], line[2]))

In [7]:
df = data_rdd.toDF(['film_id', 'ratings'])

In [8]:
df1 = df.groupBy('ratings').count()\
        .sort('ratings')

In [9]:
df2 = df.filter(df['film_id'] == film_id)\
        .groupBy('ratings').count()\
        .sort('ratings')

In [10]:
all_ratings = [row['count'] for row in df1.select('count').collect()]
my_film_ratings = [row['count'] for row in df2.select('count').collect()]

In [11]:
all_ratings, my_film_ratings

([6110, 11370, 27145, 34174, 21201], [5, 14, 46, 98, 134])

In [12]:
all_films = data_rdd.map(lambda x: (x[1],1))\
                    .reduceByKey(lambda x, y: x + y)\
                    .sortByKey()\
                    .take(5)

In [13]:
selected_film = data_rdd.filter(lambda x: x[0] == f"{film_id}")\
                        .map(lambda x: (x[1],1))\
                        .reduceByKey(lambda x, y: x + y)\
                        .sortByKey()\
                        .take(5)

In [14]:
[row[1] for row in all_films], [row[1] for row in selected_film]

([6110, 11370, 27145, 34174, 21201], [5, 14, 46, 98, 134])

In [15]:
answer = {'hist_film': my_film_ratings, 'hist_all': all_ratings}

In [17]:
import json

In [18]:
with open('lab01.json', 'w') as json_file:
    json.dump(answer, json_file)

In [ ]:
sc.stop()